In [ ]:
# Обязательно выполнять на ubuntu

### Utils

In [1]:
!pip install tqdm -q

In [1]:
import os
import requests
from tqdm import tqdm

def yandex2direct(public_url):
    # URL API
    api_url = f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_url}"

    # Получение прямой ссылки
    response = requests.get(api_url)
    direct_link = response.json().get("href")

    return direct_link

def download(filename, direct_link):
    if direct_link.startswith("https://disk.yandex.ru"):
        direct_link = yandex2direct(direct_link)

    # Создание директории, если она не существует
    dir = os.path.dirname(filename)
    if dir:
        os.makedirs(dir, exist_ok=True)

    # Стриминг данных для поэтапного сохранения
    with requests.get(direct_link, stream=True) as response:
        # Проверка успешного запроса
        response.raise_for_status()
        
        # Получаем общий размер файла
        total_size = int(response.headers.get('content-length', 0))
        
        # Запись данных в файл по частям с отображением прогресса через tqdm
        print(f'Загрузка {filename}:')
        with open(filename, "wb") as f:
            with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as pbar:
                for chunk in response.iter_content(chunk_size=8192):  # Скачивание по 8 KB
                    f.write(chunk)
                    pbar.update(len(chunk))
        print('Загрузка завершена', end="\n\n")

In [2]:
import gzip
import shutil
import zipfile

def unzip(zip_path, extract_path):
    # Создание директории, если она не существует
    if not os.path.exists(extract_path):
        os.makedirs(extract_path, exist_ok=True)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

def gunzip(gz_path, out_path=None):
    if out_path is None:
        if not gz_path.endswith('.gz'):
            raise ValueError("Файл не заканчивается на .gz, и out_path не указан.")
        out_path = gz_path[:-3]

    with gzip.open(gz_path, 'rb') as f_in:
        with open(out_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    return out_path

### Install **requirements**

In [5]:
import platform

if platform.system() == 'Linux':
    download("init.sh", "https://disk.yandex.ru/d/D3HCDHZGn440Rg")
    download("requirements.txt", "https://disk.yandex.ru/d/uEMlCEJvkinxig")

    !chmod +x init.sh
    !./init.sh
else:
    print("Этот скрипт предназначен только для Linux")

### Download files